In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import warnings
warnings.simplefilter('ignore')

import logging
logging.getLogger('statsforecast').setLevel(logging.ERROR)

# StatsForecast ⚡️

> StatsForecast offers a collection of popular univariate time series forecasting models optimized for high performance and scalability. 

## Installation

You can install `StatsForecast` with:

```python
pip install statsforecast
```

or 

```python
conda install -c conda-forge statsforecast
``` 


Vist our [Installation Guide](https://nixtla.github.io/statsforecast/examples/installation.html) for further instructions.


## Quick Start

**Minimal Example**

```python
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

sf = StatsForecast(
    models = [AutoARIMA(season_length = 12)],
    freq = 'M'
)

sf.fit(df)
sf.predict(h=12, level=[95])
```

**Get Started with this [quick guide](./examples/Getting_Started_short.ipynb).**

**Follow this [end-to-end walkthrough](./examples/Getting_Started_complete.ipynb) for best practices.**

## Why? 

Current Python alternatives for statistical models are slow, inaccurate and don't scale well. So we created a library that can be used to forecast in production environments or as benchmarks.  `StatsForecast` includes an extensive battery of models that can efficiently fit millions of time series.

## Features

* Fastest and most accurate implementations of `AutoARIMA`, `AutoETS`, `AutoCES`, `MSTL` and `Theta` in Python. 
* Out-of-the-box compatibility with Spark, Dask, and Ray.
* Probabilistic Forecasting and Confidence Intervals.
* Support for exogenous Variables and static covariates.
* Anomaly Detection.
* Familiar sklearn syntax: `.fit` and `.predict`.



## Highlights

* Inclusion of `exogenous variables` and `prediction intervals` for ARIMA.
* 20x [faster](./experiments/arima/) than `pmdarima`.
* 1.5x faster than `R`.
* 500x faster than `Prophet`. 
* 4x [faster](./experiments/ets/) than `statsmodels`.
* Compiled to high performance machine code through [`numba`](https://numba.pydata.org/).
* 1,000,000 series in [30 min](https://github.com/Nixtla/statsforecast/tree/main/experiments/ray) with [ray](https://github.com/ray-project/ray).
* Replace FB-Prophet in two lines of code and gain speed and accuracy. Check the experiments [here](https://github.com/Nixtla/statsforecast/tree/main/experiments/arima_prophet_adapter).
* Fit 10 benchmark models on **1,000,000** series in [under **5 min**](./experiments/benchmarks_at_scale/). 



Missing something? Please open an issue or write us in [![Slack](https://img.shields.io/badge/Slack-4A154B?&logo=slack&logoColor=white)](https://join.slack.com/t/nixtlaworkspace/shared_invite/zt-135dssye9-fWTzMpv2WBthq8NK0Yvu6A)


## Examples and Guides

📚 [End to End Walkthrough](https://nixtla.github.io/statsforecast/examples/getting_started_complete.html): Model training, evaluation and selection for multiple time series

🔎 [Anomaly Detection](https://nixtla.github.io/statsforecast/examples/anomalydetection.html): detect anomalies for time series using in-sample prediction intervals.

👩‍🔬 [Cross Validation](https://nixtla.github.io/statsforecast/examples/crossvalidation.html): robust model’s performance evaluation.

❄️ [Multiple Seasonalities](https://nixtla.github.io/statsforecast/examples/multipleseasonalities.html): how to forecast data with multiple seasonalities using an MSTL.

🔌 [Predict Demand Peaks](https://nixtla.github.io/statsforecast/examples/electricitypeakforecasting.html): electricity load forecasting for detecting daily peaks and reducing electric bills.

📈 [Intermittent Demand](https://nixtla.github.io/statsforecast/examples/intermittentdata.html): forecast series with very few non-zero observations. 

🌡️ [Exogenous Regressors](https://nixtla.github.io/statsforecast/examples/exogenous.html): like weather or prices





## Models

### Automatic Forecasting
Automatic forecasting tools search for the best parameters and select the best possible model for a series of time series. These tools are useful for large collections of univariate time series.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[AutoARIMA](../models.html#autoarima)|✅|✅|✅|✅|✅|
|[AutoETS](../models.html#autoets)|✅|✅|✅|✅|✅|
|[AutoCES](../models.html#autoces)|✅|✅|✅|✅|
|[AutoTheta](../models.html#autotheta)|✅|✅|✅|✅|✅|
: {tbl-colwidths="[75,25]"}

## ARIMA Family
These models exploit the existing autocorrelations in the time series.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[ARIMA](../models.html#arima)|✅|✅|✅|✅|✅|
|[AutoRegressive](../models.html#autoregressive)|✅|✅|✅|✅|✅|
: {tbl-colwidths="[75,25]"}

### Theta Family
Fit two theta lines to a deseasonalized time series, using different techniques to obtain and combine the two theta lines to produce the final forecasts.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[Theta](../models.html#theta)|✅|✅|✅|✅|✅|
|[OptimizedTheta](../models.html#optimizedtheta)|✅|✅|✅|✅|✅|
|[DynamicTheta](../models.html#dynamictheta)|✅|✅|✅|✅|✅|
|[DynamicOptimizedTheta](../models.html#dynamicoptimizedtheta)|✅|✅|✅|✅|✅|
: {tbl-colwidths="[75,25]"}


### Multiple Seasonalities
Suited for signals with more than one clear seasonality. Useful for low-frequency data like electricity and logs.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[MSTL](../models.html#mstl)|✅|✅|✅|✅|✅|
: {tbl-colwidths="[75,25]"}

### Baseline Models
Classical models for establishing baseline.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[HistoricAverage](../models.html#historicaverage)|✅|✅|✅|✅|✅|
|[Naive](../models.html#naive)|✅|✅|✅|✅|✅|
|[RandomWalkWithDrift](../models.html#randomwalkwithdrift)|✅|✅|✅|✅|✅|
|[SeasonalNaive](../models.html#seasonalnaive)|✅|✅|✅|✅|✅|
|[WindowAverage](../models.html#windowaverage)|✅|||||
|[SeasonalWindowAverage](../models.html#seasonalwindowaverage)|✅|||||
: {tbl-colwidths="[75,25]"}

### Exponential Smoothing
Uses a weighted average of all past observations where the weights decrease exponentially into the past. Suitable for data with clear trend and/or seasonality. Use the `SimpleExponential` family for data with no clear trend or seasonality.

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[SimpleExponentialSmoothing](../models.html#simpleexponentialsmoothing)|✅|||||
|[SimpleExponentialSmoothingOptimized](../models.html#simpleexponentialsmoothingoptimized)|✅|||||
|[SeasonalExponentialSmoothing](../models.html#seasonalexponentialsmoothing)|✅|||||
|[SeasonalExponentialSmoothingOptimized](../models.html#seasonalexponentialsmoothingoptimized)|✅|||||
|[Holt](../models.html#holt)|✅|✅|✅|✅|✅|
|[HoltWinters](../models.html#holtwinters)|✅|✅|✅|✅|✅|
: {tbl-colwidths="[75,25]"}




### Sparse of Inttermitent
Suited for series with very few non-zero observations

|Model | Point Forecast | Probabilistic Forecast | Insample fitted values | Probabilistic fitted values |
|:------|:-------------:|:----------------------:|:---------------------:|:----------------------------:|
|[ADIDA](../models.html#adida)|✅|||||
|[CrostonClassic](../models.html#crostonclassic)|✅|||||
|[CrostonOptimized](../models.html#crostonoptimized)|✅|||||
|[CrostonSBA](../models.html#crostonsba)|✅|||||
|[IMAPA](../models.html#imapa)|✅|||||
|[TSB](../models.html#tsb)|✅|||||
: {tbl-colwidths="[75,25]"}



## How to contribute
See [CONTRIBUTING.md](https://github.com/Nixtla/statsforecast/blob/main/CONTRIBUTING.md).

## Citing

```bibtex
@misc{garza2022statsforecast,
    author={Federico Garza, Max Mergenthaler Canseco, Cristian Challú, Kin G. Olivares},
    title = {{StatsForecast}: Lightning fast forecasting with statistical and econometric models},
    year={2022},
    howpublished={{PyCon} Salt Lake City, Utah, US 2022},
    url={https://github.com/Nixtla/statsforecast}
}
```